In [2]:
import warnings
import os
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from tensorflow.keras.models import load_model
import tensorflow.keras as keras
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

2024-05-29 19:46:26.205817: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 19:46:26.323092: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
gpus = tf.config.experimental.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
print('\n\n Running on multiple GPUs ', [gpu.name for gpu in gpus])

2024-05-29 19:46:36.953225: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 19:46:38.605565: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14825 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:3b:00.0, compute capability: 7.5
2024-05-29 19:46:38.606657: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14825 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:5e:00.0, compute capability: 7.5
2024-05-29 19:46:38.607651: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:local



 Running on multiple GPUs  ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3']


In [4]:
with strategy.scope():
    w, h = 19, 19
    base_path = os.path.join('dataset2', '*/*.npz')
    
    file_list = glob(base_path)
    
    x_data, y_data = [], []
    chk = 0
    for file_path in tqdm(file_list):
        chk += 1
        if chk > 60001:  break
        data = np.load(file_path)
        x_data.extend(data['inputs'])
        y_data.extend(data['outputs'])
    
    x_data = np.array(x_data, np.float32).reshape((-1, h, w, 1))
    y_data = np.array(y_data, np.float32).reshape((-1, h * w))
    
    x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=2020)
    
    del x_data, y_data
    
    print(x_train.shape, y_train.shape)
    print(x_val.shape, y_val.shape)

100%|████████████████████████████████████| 24938/24938 [00:34<00:00, 722.95it/s]


(239404, 19, 19, 1) (239404, 361)
(59852, 19, 19, 1) (59852, 361)


In [6]:
with strategy.scope():
    existing_model = load_model('/home/jinjinjara1022/OmokAI/models/best.h5')
    
    start_time = datetime.now().strftime('%Y%m%d_%H%M%S')
    os.makedirs('models', exist_ok=True)
    
    existing_model.fit(
        x=x_train,
        y=y_train,
        batch_size=128,
        epochs=10,
        callbacks=[
            ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
            ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, verbose=1, mode='auto')
        ],
        validation_data=(x_val, y_val),
        use_multiprocessing=True,
        workers=16
    )

Epoch 1/10


2024-05-29 19:49:45.361660: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8907
2024-05-29 19:49:46.170486: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8907
2024-05-29 19:49:47.032362: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8907
2024-05-29 19:49:47.434456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8907
2024-05-29 19:49:47.441190: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-29 19:49:47.444521: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2024-05-29 19:49:4

1871/1871 [==============================] - ETA: 0s - loss: 5.7549e-04 - acc: 0.9635
Epoch 1: val_acc improved from -inf to 0.97036, saving model to ./models/20240529_194939.h5
1871/1871 [==============================] - 67s 30ms/step - loss: 5.7549e-04 - acc: 0.9635 - val_loss: 4.6696e-04 - val_acc: 0.9704 - lr: 0.0010
Epoch 2/10
1869/1871 [============================>.] - ETA: 0s - loss: 3.2164e-04 - acc: 0.9814
Epoch 2: val_acc improved from 0.97036 to 0.97420, saving model to ./models/20240529_194939.h5
1871/1871 [==============================] - 53s 28ms/step - loss: 3.2157e-04 - acc: 0.9814 - val_loss: 4.7192e-04 - val_acc: 0.9742 - lr: 0.0010
Epoch 3/10
  13/1871 [..............................] - ETA: 48s - loss: 2.0688e-04 - acc: 0.9874

KeyboardInterrupt: 